In [22]:
import pandas as pd

In [23]:
game = pd.read_parquet("G:\Mi unidad\BOOTCAMPS\SOYHENRY\CIENCIA DE DATOS\CURSO\PT07\PI_MLOPS\DATA_LIMPIA\gameclean.parquet")
item = pd.read_parquet("G:\Mi unidad\BOOTCAMPS\SOYHENRY\CIENCIA DE DATOS\CURSO\PT07\PI_MLOPS\DATA_LIMPIA\itemclean.parquet")
review = pd.read_parquet("G:\Mi unidad\BOOTCAMPS\SOYHENRY\CIENCIA DE DATOS\CURSO\PT07\PI_MLOPS\DATA_LIMPIA//reviewclean.parquet")

<h1><strong><em>PlayTimeGenre</em></strong></h1>
<p>def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.<br>
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}</p>

In [24]:

# Función para explotar dos columnas simultáneamente
def explode_columns(df, col1, col2, extra_col):
    # Combinar las listas de ambas columnas utilizando zip y explotarlas
    exploded = df.apply(lambda row: list(zip(row[col1], row[col2], [row[extra_col]] * len(row[col1]))), axis=1).explode()
    # Convertir la serie resultante en un DataFrame
    exploded_df = exploded.apply(pd.Series)
    # Asignar nombres de columnas
    exploded_df.columns = [col1, col2, extra_col]
    return exploded_df.reset_index(drop=True)

In [25]:
df_item = explode_columns(item, 'item_id', 'playtime_forever', 'user_id')
df_item.head(30)

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
0,20,0.0,76561197970982479
0,30,7.0,76561197970982479
0,40,0.0,76561197970982479
0,50,0.0,76561197970982479
0,60,0.0,76561197970982479
0,70,0.0,76561197970982479
0,130,0.0,76561197970982479
0,300,4733.0,76561197970982479
0,240,1853.0,76561197970982479


In [26]:
# Modificaciones a los dataframes para mejorar el rendimiento de las funciones
playtiem_item = df_item.groupby('item_id')['playtime_forever'].sum().reset_index()
df_merge = pd.merge(game, playtiem_item, on='item_id', how='inner')
df_PlayTimeGenre = df_merge[["genres","year","playtime_forever"]]
df_PlayTimeGenre =df_PlayTimeGenre.dropna()


In [27]:
def PlayTimeGenre(genero : str):
    # Filtrar el DataFrame para obtener solo las filas que contienen el género específico
    filtro_genero = df_PlayTimeGenre['genres'].str.contains(genero, case=False, na=False)
    df_filtrado = df_PlayTimeGenre[filtro_genero]
    df_agrupado = df_filtrado.groupby('year')['playtime_forever'].sum().reset_index()
    # Encontrar el año con la mayor suma de 'playtime_forever'
    df_agrupado['year'] = df_agrupado['year'].astype(int)
    anio_mayor_suma = df_agrupado.loc[df_agrupado['playtime_forever'].idxmax(), 'year']
    respuesta = {f"Año de lanzamiento con más horas jugadas para Género el {genero}": anio_mayor_suma}
    return respuesta 

PlayTimeGenre("action")

{'Año de lanzamiento con más horas jugadas para Género el action': 2012}

In [28]:
#funciona correctamente, asi que guardamos el dataframe en un archivo parquet
df_PlayTimeGenre.to_parquet('df_PlayTimeGenre.parquet',index=False)

<h1><strong><em>UserForGenre</em></strong></h1>
<p>def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año. <br>
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}</p>

In [29]:
df_aux = game[["item_id","genres","year"]]
df_UserForGenre = pd.merge(df_item,df_aux, on = "item_id")
df_UserForGenre =df_UserForGenre[["user_id","genres","year","playtime_forever"]]
def eliminar_outliers_iqr(df, columna):
    # Calcular el rango intercuartílico (IQR)
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1

    # Definir los límites superior e inferior para identificar outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Filtrar el DataFrame para mantener solo los valores dentro de los límites
    df_filtrado = df[(df[columna] >= limite_inferior) & (df[columna] <= limite_superior)]

    return df_filtrado

# Ejemplo de uso
# Supongamos que 'df' es tu DataFrame y 'valor' es la columna que contiene los datos
df_UserForGenre = eliminar_outliers_iqr(df_UserForGenre, 'playtime_forever')


In [30]:
def userforgenre(genero: str):
    # Filtrar el DataFrame para el género especificado
    filtro_genero = df_UserForGenre['genres'].str.contains(genero, case=False, na=False)
    df_filtrado = df_UserForGenre[filtro_genero]

    # Agrupar por 'user_id' y 'year', sumar las horas jugadas
    df_agrupado = df_filtrado.groupby(['user_id', 'year'])['playtime_forever'].sum().reset_index()

    # Encontrar el usuario con la máxima suma de horas jugadas
    idx_max_playtime = df_agrupado['playtime_forever'].idxmax()
    usuario_max_playtime = df_agrupado.loc[idx_max_playtime, 'user_id']

    # Filtrar el DataFrame para el usuario con máxima suma de horas jugadas
    df_usuario = df_agrupado[df_agrupado['user_id'] == usuario_max_playtime]

    # Crear el formato "Horas jugadas"
    resultado_final = [{'Año': int(row['year']), 'Horas': int(row['playtime_forever'])} for _, row in df_usuario.iterrows()]
    
    return {"Usuario con más horas jugadas para Género {}:".format(genero): usuario_max_playtime, "Horas jugadas": resultado_final}

# Ejemplo de uso
userforgenre('actIon')

{'Usuario con más horas jugadas para Género actIon:': 'phrostb',
 'Horas jugadas': [{'Año': 1988, 'Horas': 92},
  {'Año': 1989, 'Horas': 0},
  {'Año': 1990, 'Horas': 0},
  {'Año': 1991, 'Horas': 0},
  {'Año': 1992, 'Horas': 0},
  {'Año': 1993, 'Horas': 1384},
  {'Año': 1994, 'Horas': 623},
  {'Año': 1995, 'Horas': 931},
  {'Año': 1996, 'Horas': 589},
  {'Año': 1997, 'Horas': 1005},
  {'Año': 1998, 'Horas': 202},
  {'Año': 1999, 'Horas': 512},
  {'Año': 2000, 'Horas': 635},
  {'Año': 2001, 'Horas': 400},
  {'Año': 2002, 'Horas': 737},
  {'Año': 2003, 'Horas': 687},
  {'Año': 2004, 'Horas': 375},
  {'Año': 2005, 'Horas': 1155},
  {'Año': 2006, 'Horas': 1396},
  {'Año': 2007, 'Horas': 1283},
  {'Año': 2008, 'Horas': 331},
  {'Año': 2009, 'Horas': 3960},
  {'Año': 2010, 'Horas': 2888},
  {'Año': 2011, 'Horas': 5148},
  {'Año': 2012, 'Horas': 14150},
  {'Año': 2013, 'Horas': 24348},
  {'Año': 2014, 'Horas': 53119},
  {'Año': 2015, 'Horas': 125210},
  {'Año': 2016, 'Horas': 66941},
  {'Año':

In [31]:
#funciona correctamente, entonces exportamos el dataframe a un archivo parquet
df_UserForGenre.to_parquet(r'df_UserForGenre.parquet',index=False)

<h1><strong><em>UsersRecommend</em></strong></h1>
<p>def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]</p>

In [39]:
#Función para explotar 4 columnas simultanemanete
def explode_columns(df, col1, col2, col3, col4):
    # Combinar las listas de ambas columnas utilizando zip y explotarlas
    exploded = df.apply(lambda row: list(zip(row[col1], row[col2], row[col3], row[col4] )), axis=1).explode()
    # Convertir la serie resultante en un DataFrame
    exploded_df = exploded.apply(pd.Series)
    # Asignar nombres de columnas
    exploded_df.columns = [col1, col2, col3, col4]
    return exploded_df.reset_index(drop=True)

In [40]:
df_review = explode_columns(review, 'item_id', 'recommend', 'year', 'sentiment_analysis')
df_review.head(30)

,item_id,recommend,year,sentiment_analysis
0,1250,True,2011,2.0
1,22200,True,2011,2.0
2,43110,True,2011,2.0
3,251610,True,2014,2.0
4,227300,True,2013,2.0
5,239030,True,2013,2.0
6,248820,True,None,2.0
7,370360,True,2015,2.0
8,237930,True,2014,2.0
9,263360,True,2014,1.0


In [68]:
df_aux= game[["item_id","app_name"]]
df_UserRecommend = pd.merge(df_review,df_aux, on="item_id", how="inner")
df_UserRecommend = df_UserRecommend[df_UserRecommend['recommend'] == True]
df_UserRecommend = df_UserRecommend[['app_name','year', 'sentiment_analysis']]


In [110]:
def UsersRecommend(año: int):
    
    df = df_UserRecommend[df_UserRecommend['year'] == str(año)]

    # Contar el número de recomendaciones para cada juego
    top_juegos = df[['sentiment_analysis', 'app_name']].groupby('app_name').sum().sort_values(by='sentiment_analysis', ascending=False).head(3)

    # Crear el formato de salida
    resultado_final = [{"Puesto {}".format(i+1): juego} for i, (juego, _) in enumerate(top_juegos.reset_index().values)]

    return str(resultado_final)

# Ejemplo de uso
UsersRecommend(2012)

'[{\'Puesto 1\': \'Team Fortress 2\'}, {\'Puesto 2\': \'Terraria\'}, {\'Puesto 3\': "Garry\'s Mod"}]'

In [103]:
#funciona correctamente, entonces exportamos el dataframe a un archivo parquet
df_UserRecommend = df_UserRecommend.dropna()
df_UserRecommend.to_parquet(r'df_UserRecommend.parquet',index=False)

<h1><strong><em>UsersNotRecommend</em></strong></h1>
<p>def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]</p>

In [75]:
df_info = game[["item_id","app_name"]]
df_UsersNotRecommend  = pd.merge(df_review,df_info, on ="item_id",how= "inner")
df_UsersNotRecommend = df_UsersNotRecommend[(df_UsersNotRecommend['recommend'] == False) & (df_UsersNotRecommend['sentiment_analysis'] == 0)]
df_UsersNotRecommend = df_UsersNotRecommend[['app_name','year']]


In [87]:
def UsersNotRecommend(año: int):
    
    df = df_UsersNotRecommend[df_UsersNotRecommend['year'] == str(año)]

    # Contar el número de recomendaciones para cada juego
    top_juegos = df['app_name'].value_counts().head(3)

    # Crear el formato de salida
    resultado_final = [{"Puesto {}".format(i+1): juego} for i, (juego, _) in enumerate(top_juegos.reset_index().values)]

    return resultado_final

# Ejemplo de uso
UsersNotRecommend(2012)

[{'Puesto 1': 'Resident Evil: Operation Raccoon City'},
 {'Puesto 2': 'Red Faction®: Armageddon™'},
 {'Puesto 3': 'Team Fortress 2'}]

In [104]:
#funciona correctamente, entonces exportamos el dataframe a un archivo parquet
df_UsersNotRecommend = df_UsersNotRecommend.dropna()
df_UsersNotRecommend.to_parquet(r'df_UsersNotRecommend.parquet',index=False)

<h1><strong><em>Sentiment_analysis</em></strong></h1>
<p>def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}</p>

In [97]:
df_10 = df_review[["item_id","sentiment_analysis"]]
df_11 = game[["year","item_id"]]
df_Sentiment_Analysis = pd.merge(df_10,df_11, on = "item_id", how= "inner") 
df_Sentiment_Analysis = df_Sentiment_Analysis.groupby(['year', 'sentiment_analysis']).size().unstack(fill_value=0)
df_Sentiment_Analysis = df_Sentiment_Analysis.reset_index(drop=False)
nuevos_nombres = ["year",'Negativo', 'Neutro',"Positivo"]
df_Sentiment_Analysis.columns = nuevos_nombres
df_Sentiment_Analysis["year"] = df_Sentiment_Analysis["year"].astype(str).str.lower()


In [106]:
df_Sentiment_Analysis = df_Sentiment_Analysis.dropna()
df_Sentiment_Analysis.head(10)

,year,Negativo,Neutro,Positivo
0,1989,0,0,1
1,1990,1,0,4
2,1991,0,0,1
3,1992,1,0,2
4,1993,1,0,4
5,1994,3,3,9
6,1995,5,0,11
7,1996,0,2,5
8,1997,8,7,33
9,1998,16,16,63


In [108]:
def Sentiment_analysis(año: int):
    # Filtra el DataFrame para obtener la fila correspondiente al año de lanzamiento
    
    year_fila = df_Sentiment_Analysis[df_Sentiment_Analysis['year'] == str(año)]

    # Extrae los valores de las columnas Negativo, Neutro y Positivo
    negativo = year_fila['Negativo'].values[0]
    neutro = year_fila['Neutro'].values[0]
    positivo = year_fila['Positivo'].values[0]

    # Crea un diccionario con la información en el formato solicitado
    resultado = {
        año: {
            'Negative ': negativo,
            'Neutral': neutro,
            'Positive': positivo
        }
    }

    return resultado
Sentiment_analysis(1998)

{1998: {'Negative ': 16, 'Neutral': 16, 'Positive': 63}}

In [109]:
#funciona correctamente, entonces exportamos el dataframe a un archivo parquet
df_Sentiment_Analysis.to_parquet("df_Sentiment_Analysis.parquet", index=False )